In [13]:
# importing library
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [14]:
client = pymongo.MongoClient('localhost',27017)
my_database = client['My_Database']

customer_info = my_database.customer

In [15]:
#inserting record
record1 = {
    'name':'Mohan Swami',
    'age':32,
    'gender':'Male',
    'date joined':'2019-09-30T09:18:29.044Z',
    'state':'MH',
    'country':'IN'
}
# here I created only one record
# inserting data value

customer_info.insert_one(record1)

# To insert multiple data value than we need create a list and using insert_many() to insert

In [16]:
# Lets create farm collection and device collection as well
farm_info = my_database.farm

record2 = {
    'customer':'614afecbead14a04475de1b7',
    'name':'tomato_farm',
    'crop':'tomato',
    'device_id':"7HGRXY",
    'lat':20.632189,
    'long':74.8961
}

farm_info.insert_one(record2)

device_info = my_database.device

record3={
    'device_id':'7HGRXY',
    'TC':24,
    'HUM':84,
    'datetime':'2021-07-30T09:18:29.044Z',
    'LW':0,
    'PLV2':0
}

device_info.insert_one(record3)

In [58]:
customer_df=pd.DataFrame(list(customer_info.find()))
farm_df=pd.DataFrame(list(farm_info.find()))
device_df=pd.DataFrame(list(device_info.find()))

In [62]:
# merging customer_info into farm_info 
merge_df=pd.merge(customer_df,farm_df,how="inner",left_on="_id",right_on="customer")

# merging resultant dataframe into device_info
get_df=pd.merge(merge_df,device_df,how="inner",on="device_id")
get_df.head().T

,0,1,2,3,4
_id_x,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7
name_x,Mohan Swami,Mohan Swami,Mohan Swami,Mohan Swami,Mohan Swami
age,32,32,32,32,32
gender,Male,Male,Male,Male,Male
date joined,2019-09-30T09:18:29.044Z,2019-09-30T09:18:29.044Z,2019-09-30T09:18:29.044Z,2019-09-30T09:18:29.044Z,2019-09-30T09:18:29.044Z
state,MH,MH,MH,MH,MH
country,IN,IN,IN,IN,IN
_id_y,614c506f807151598a045be6,614c506f807151598a045be6,614c506f807151598a045be6,614c90b00598f3fd76cf1f61,614c90b00598f3fd76cf1f61
customer,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7,614afecbead14a04475de1b7
name_y,tomato_farm,tomato_farm,tomato_farm,tomato_farm,tomato_farm


In [64]:
get_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id_x        6 non-null      object 
 1   name_x       6 non-null      object 
 2   age          6 non-null      int64  
 3   gender       6 non-null      object 
 4   date joined  6 non-null      object 
 5   state        6 non-null      object 
 6   country      6 non-null      object 
 7   _id_y        6 non-null      object 
 8   customer     6 non-null      object 
 9   name_y       6 non-null      object 
 10  crop         6 non-null      object 
 11  device_id    6 non-null      object 
 12  lat          6 non-null      float64
 13  long         6 non-null      float64
 14  _id          6 non-null      object 
 15  TC           6 non-null      int64  
 16  HUM          6 non-null      int64  
 17  datetime     6 non-null      object 
 18  LW           6 non-null      int64  
 19  PLV2        

In [147]:
# creating function to extract data of every hour
def get_data(cropname,start_date,end_date,sensor_list):
    day_df = get_df.loc[(get_df["datetime"]>=start_date) & (get_df["datetime"]<end_date) & (get_df["crop"]==cropname)]
    hour_df = day_df.groupby(["_id_y","crop","device_id",day_df['datetime'].dt.hour]).sum()
    hour_df.drop(['lat','long','age'],axis=1,inplace=True)
    return hour_df
    

sensor_list=["TC","HUM","LW","PLV2"]

# Creating a datetime object
get_df['datetime'] = pd.to_datetime(get_df['datetime'])
date_list=get_df["datetime"].unique()

#creating list of crops
crop_list=get_df["crop"].unique()

for i in crop_list:
    cropname=i
    for i in date_list:
        start_date=i
        end_date=i+timedelta(1)
        resultant_df = get_data(cropname,start_date,end_date,sensor_list)
resultant_df.head()

,,,,TC,HUM,LW,PLV2
_id_y,crop,device_id,datetime,,,,
614c506f807151598a045be6,tomato,7HGRXY,9,72,252,0,0
614c90b00598f3fd76cf1f61,tomato,7HGRXY,9,72,252,0,0


In [148]:
resultant_df.to_csv("required_csv.csv")